In [1]:
import jax
import jax.numpy as jnp

from flax import linen as nn
from flax.training import orbax_utils
import orbax
from jaxlie import SO3, SE3

In [11]:
# models 
orbax_checkpointer = orbax.checkpoint.PyTreeCheckpointer()
restored_grasp = orbax_checkpointer.restore("model/grasp_net_waffle")

class GraspNet(nn.Module):
    hidden_dim: int
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=self.hidden_dim)(x)
        x = nn.relu(x)
        x = nn.Dense(features=self.hidden_dim)(x)
        x = nn.relu(x)
        x = nn.Dense(features=self.hidden_dim)(x)
        x = nn.relu(x)
        logit = nn.Dense(features=5)(x)
        return logit
    
#grasp net
grasp_net = GraspNet(32)
grasp_fn = lambda x: grasp_net.apply(restored_grasp["params"], x)

grasp_logit_fn = lambda g: grasp_fn(g)[0]
def grasp_reconst(grasp):
    rot = SO3(grasp_fn(grasp)[1:]).normalize()
    trans = grasp/restored_grasp["scale_to_norm"]
    return SE3.from_rotation_and_translation(rot, trans)

In [12]:
grasp_reconst(jnp.zeros(3))
grasp_fn(jnp.zeros(3))

SE3(wxyz=[-0.02453  0.87692  0.21851 -0.42738], xyz=[0. 0. 0.])

: 

Array([-130.35942   ,   -0.13585927,    4.856992  ,    1.2102599 ,
         -2.367128  ], dtype=float32)